In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from splinter import Browser

## Step 1 - Scraping

### NASA Mars News

In [2]:
# URL of NASA news
url = 'https://mars.nasa.gov/news/'

# Retrieve page with the requests module
html = requests.get(url)

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html.text, 'html.parser')
#list(soup.children)
#display(soup.prettify())

 
# Examine the results, then determine element that contains sought info
# results are returned as an iterable list
results = soup.find_all('div', class_= "slide")[0]

# display(results.prettify())

news_p_list = []
news_title_list = []
for div in soup.find_all('div', class_= "slide"):
    news_p_list.append(div.find('div', attrs={'class': 'rollover_description_inner'}).text)
    news_title_list.append(div.find('div', attrs={'class': 'content_title'}).text)
    
news_title_list = [w.replace('\n', '') for w in news_title_list]
news_p_list = [w.replace('\n', '') for w in news_p_list]
news_title = news_title_list[0]
news_p = news_p_list[0]
display(news_title)
display(news_p)

'A Piece of Mars is Going Home'

"When it launches in 2020, NASA's next Mars rover will carry a chunk of Martian meteorite on board."

### JPL Mars Space Images - Featured Image

In [11]:
browser = Browser('chrome', headless=False)
pic_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(pic_url)

#  Click & pull new page html
browser.click_link_by_partial_text('FULL IMAGE')
pic_html = browser.html

# Parse new html
soup = BeautifulSoup(pic_html, 'html.parser')

# Image URL
partial = soup.find('a', class_ = 'button fancybox')['data-fancybox-href']
featured_image_url = "https://www.jpl.nasa.gov" + partial

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA10181_ip.jpg


### Mars Weather

In [12]:
# URL of NASA tweets
url = 'https://twitter.com/marswxreport?lang=en'

# Retrieve page with the requests module
html = requests.get(url)

soup = BeautifulSoup(html.text, 'html.parser')
results = soup.find_all('div', class_= "stream")

for result in results:
    if result.ol.li.div['data-name'] == "Mars Weather":
        mars_weather = (result.p.text)

display(mars_weather)

'Sol 1962 (Feb 12, 2018), Sunny, high -14C/6F, low -78C/-108F, pressure at 7.38 hPa, daylight 05:40-17:27'

### Mars Facts

In [6]:
# URL of Mars facts
url = 'https://space-facts.com/mars/'

# Retrieve page with the requests module
html = requests.get(url)

soup = BeautifulSoup(html.text, 'html.parser')

td1 = []
td2 = []

for tr in soup.find_all('tr'):	
    td1.append(tr.find_all('td')[0].text.replace('\n',''))
    td2.append(tr.find_all('td')[1].text.replace('\n',''))
    
df = pd.DataFrame(
    {'column_name': td1,
     'stat': td2
    })
mars_html = df.to_html
display(mars_html)

<bound method DataFrame.to_html of              column_name                           stat
0   Equatorial Diameter:                       6,792 km
1        Polar Diameter:                       6,752 km
2                  Mass:  6.42 x 10^23 kg (10.7% Earth)
3                 Moons:            2 (Phobos & Deimos)
4        Orbit Distance:       227,943,824 km (1.52 AU)
5          Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -153 to 20 °C
7          First Record:              2nd millennium BC
8           Recorded By:           Egyptian astronomers>

### Mars Hemispheres

In [10]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', headless=False)
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
# Retrieve page with the requests module
html = requests.get(url)

soup = BeautifulSoup(html.text, 'html.parser')
results = soup.find_all('h3')
titles = []
for result in results:
    titles.append(result.text)
#display(titles)

title = []
img_url = []

for x in titles:
    browser.visit(url)
    browser.click_link_by_partial_text(x)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    results = soup.find('div', class_= "downloads")
    title.append(x)
    img_url.append(results.ul.li.a['href'])

mars_dict = dict(zip(title, img_url))
display(mars_dict)


['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced']

{'Cerberus Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'Schiaparelli Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'Syrtis Major Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'Valles Marineris Hemisphere Enhanced': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}

## Step 2 - MongoDB and Flask Application